Este dataset chamado "possum(gambá)" contém nove medidas morfométricas para 104 gambás da montanha. Esses gambás foram capturados em sete locais diferentes, do sul de Victoria ao centro de Queensland. Nele iremos utilizar de Regressao Linear, Regressao Logistica e Random Forest para tentar predizer qual o tamanho da cabeca de um determinado gambá.

Existem 14 variáveis ​​​​no conjunto de dados:
* Caso (número do gambá, ID exclusivo)
* Local da gravação
* Área populacional
* Sexo
* Idade
* Comprimento da cabeça
* Largura do crânio
* Comprimento total
* Comprimento da cauda
* Comprimento do pé
* Comprimento da concha da orelha
* Tamanho dos olhos
* Peito
* Barriga